In [1]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from keras.models import model_from_json

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM
import keras.layers as lyr
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from keras.layers import Bidirectional
import tensorflow as tf


Using TensorFlow backend.


In [2]:
TRAIN_CSV = 'questions.csv'
#TRAIN_CSV = 'sample.csv'
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin'


# In[15]:

train_df = pd.read_csv(TRAIN_CSV)
stops = set(stopwords.words('english'))


# In[16]:

def stringToWordList(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\(", " ( ", text)
    text = re.sub(r"\)", " ) ", text)
    text = re.sub(r"\?", " ? ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = text.split()
    return text


# In[17]:

questions_cols = ['question1', 'question2']
vocab = dict()
maxSeqLength = 0

In [3]:
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

for index, row in train_df.iterrows():
    for question in questions_cols:
        q2num = []
        for word in stringToWordList(row[question]):
            if word in stops and word not in word2vec.vocab:
                continue
            if word not in vocab:
                vocab[word] = len(vocab)+1
            q2num.append(vocab[word])
        train_df.set_value(index,question,q2num)
        if len(q2num) > maxSeqLength:
            maxSeqLength = len(q2num)

embeddingDim = 300
embeddings = 1 * np.random.randn(len(vocab) + 1, embeddingDim)
embeddings[0] = 0

C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':


In [4]:
train_df.head()

,Unnamed: 0,id,qid1,qid2,question1,question2,is_duplicate
0,0,0,1,2,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 8, 11]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10]",0
1,1,1,3,4,"[1, 2, 3, 12, 13, 14, 15, 16, 15, 17, 18]","[1, 19, 20, 21, 3, 22, 23, 24, 3, 13, 14, 15, ...",0
2,2,2,5,6,"[26, 27, 16, 28, 3, 29, 30, 31, 32, 33, 34, 35]","[26, 27, 31, 29, 36, 37, 5, 38, 39, 40]",0
3,3,3,7,8,"[41, 42, 16, 43, 44, 45, 26, 27, 16, 46, 47]","[48, 3, 49, 50, 51, 52, 53, 54, 51, 2, 55, 5, ...",0
4,4,4,9,10,"[56, 57, 58, 8, 59, 60, 61, 62, 63, 64, 65, 66]","[56, 67, 19, 68, 8, 62, 59]",0


In [5]:
del word2vec

In [6]:
test_size = 20000
validation_size = 40000
training_size = len(train_df) - validation_size

X = train_df[questions_cols]
Y = train_df['is_duplicate']

X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=test_size)
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train_val, Y_train_val, test_size=validation_size)

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_test = {'left': X_test.question1, 'right': X_test.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_test = Y_test.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation, X_test], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=maxSeqLength)

# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

In [7]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 1

# The visible layer
left_input = Input(shape=(maxSeqLength,), dtype='int32')
right_input = Input(shape=(maxSeqLength,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embeddingDim, weights=[embeddings], input_length=maxSeqLength, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = Bidirectional(LSTM(n_hidden))

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

W1009 12:32:55.383420 79092 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1009 12:32:56.883743 79092 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1009 12:32:58.023118 79092 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1009 12:32:58.710837 79092 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Pl

In [8]:
# Concatenate the embeddings with their product and squared difference.
p = lyr.multiply([left_output, right_output])
negative_right_output = lyr.Lambda(lambda x: -x)(right_output)
d = lyr.add([left_output, negative_right_output])
q = lyr.multiply([d, d])
v = [left_output, right_output, p, q]
lstm_output = lyr.concatenate(v)

merged = lyr.BatchNormalization()(lstm_output)
merged = lyr.Dense(64, activation='relu')(merged)
merged = lyr.Dropout(0.2)(merged)
merged = lyr.BatchNormalization()(merged)
preds = lyr.Dense(1, activation='sigmoid')(merged)
model = Model(input=[left_input,right_input], output=preds)
optimizer = Adadelta(clipnorm=gradient_clipping_norm)
model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['accuracy'])


W1009 12:33:08.160382 79092 deprecation.py:506] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3733: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  
W1009 12:33:08.276046 79092 deprecation_wrapper.py:119] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
training_start_time = time()

lstm_trained = model.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size,epochs=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

W1009 12:33:20.974612 79092 deprecation.py:323] From C:\Users\20131073\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 344351 samples, validate on 40000 samples
Epoch 1/1
344351/344351 [==============================] - 2787s 8ms/step - loss: 0.1615 - acc: 0.7647 - val_loss: 0.1422 - val_acc: 0.7956
Training time finished.
1 epochs in 0:46:29.620206


In [ ]:
###https://github.com/Dhawnit/Duplicate-Question-Pair-Detection-using-Deep-Learning/blob/master/nlpProj.py

In [14]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
scores =model.evaluate([X_test['left'], X_test['right']], Y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

acc: 79.55%


In [33]:
X_test['left']

array([[   0,    0,    0, ..., 5658, 8721, 2496],
       [   0,    0,    0, ...,  258, 4470,  518],
       [   0,    0,    0, ...,   84,  212,  781],
       ...,
       [   0,    0,    0, ..., 1267,    8, 9667],
       [   0,    0,    0, ...,    2,   83, 7840],
       [   0,    0,    0, ...,  191,   47, 3570]])